In [1]:
import os

# Use os.chdir() to change the directory
os.chdir(r"C:\Users\Welcome\Desktop\medi-chatbot-")

# Verify that the directory has been changed
print("Current Directory:", os.getcwd())


Current Directory: C:\Users\Welcome\Desktop\medi-chatbot-


In [2]:
%pwd

'C:\\Users\\Welcome\\Desktop\\medi-chatbot-'

In [3]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
#Extract Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_file(data = 'Data/')

In [6]:
#extracted_data


In [6]:
#chunking operation
# split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20) 
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 5860


In [8]:
#download the embeddings from the hugging face
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\Welcome\AppData\Local\Temp\ipykernel_15624\3657163953.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Welcome\Desktop\medi-chatbot-\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#cheking here embedding hua download huahai ya nhi and checking their dimension
query_result = embeddings.embed_query("hello world")
print("Length", len(query_result))


Length 384


In [42]:
#checking vector
#query_result

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.environ.get('HUGGINGFACEHUB_API_TOKEN')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

In [15]:
#initialize pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medi-chatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


In [16]:
# Set your Hugging Face API key and pinecone 
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN 
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [17]:
# Embed each chunk and upsert the embeddings into your pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [18]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [19]:
docsearch

In [20]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [21]:
retrieved_docs = retriever.invoke("what is fever")

In [22]:
retrieved_docs
#this dont give the complete answer that's why we need to llm 

[Document(id='0f7ebe5b-7017-48f0-a8ef-125b4bf22906', metadata={'page': 69.0, 'source': 'Data\\Medical_book.pdf'}, page_content='fever in children. This disease is most often caused by\ntypes 3 and 7. Symptoms, which appear suddenly and\nusually disappear in less than a week, include:\n• inflammation of the lining of the eyelid (conjunctivitis)\n•f e v e r\n• sore throat (pharyngitis)\n• runny nose\n• inflammation of lymph glands in the neck (cervical\nadenitis)\nGALE ENCYCLOPEDIA OF MEDICINE 256\nAdenovirus infections\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 56'),
 Document(id='5533c259-039f-49fa-825d-83f2cdba19b6', metadata={'page': 60.0, 'source': 'Data\\Medical_book.pdf'}, page_content='(38°–40°C). In addition, a general ill feeling, muscle\naches,headache, chills, and loss of appetite may be felt.\nDiagnosis\nIf lymphangitis is suspected, the person should call\nhis or her doctor immediately or go to an emergency\nroom. Acute lymphangitis could be diagnosed by the\nfamily do

In [23]:
import google.generativeai as genai

# Configure the Gemini API key
genai.configure(api_key=GEMINI_API_KEY)  # Replace with your Gemini API key

# System prompt
system_prompt = (
    "You are a highly knowledgeable assistant tasked with providing detailed and accurate answers. "
    "For the given query, please answer in detail, covering all aspects of the topic. "
    "If the query asks for multiple components (e.g., causes, symptoms, treatments), provide a thorough breakdown for each. "
    "Be as detailed as possible within the token limit, and ensure that you give examples where applicable. "
    "Always reference the retrieved context when answering, and if more information is needed, request clarification politely. "
    "If the information in the context is insufficient, clearly state that and ask for further details."
    "\n\n{context}"
)

# Initialize the Gemini model
llm = genai.GenerativeModel("gemini-1.5-flash")

# Define a function for generating responses
def generate_response(llm, prompt, context, input_text):
    # Combine system prompt and input
    full_prompt = prompt.format(context=context) + "\n\n" + input_text
    # Generate content without additional arguments
    response = llm.generate_content(full_prompt)
    return response.text

# Simulate retrieval (mock context retrieval)
def mock_retriever(query):
    return "Mocked retrieved context for: " + query

# Generate the response
retrieved_context = mock_retriever("what is fever?")
response_text = generate_response(llm, system_prompt, retrieved_context, "what is fever?")
print(response_text)


Fever, or pyrexia, is a temporary increase in the body's temperature above the normal range, typically considered to be 98.6°F (37°C) orally.  It's not a disease itself but rather a symptom of an underlying condition, often indicating that the body's immune system is fighting off an infection or other illness.  The body's thermostat, located in the hypothalamus region of the brain, is responsible for regulating temperature.  When the body detects a threat, like bacteria or a virus, the hypothalamus raises the set point, leading to a higher body temperature.

**Causes of Fever:**

Fevers are caused by a wide range of factors, primarily infections but also including other conditions.  These include:

* **Infections:** This is the most common cause.  Viral infections (like the flu, common cold, or COVID-19), bacterial infections (like strep throat, pneumonia, or urinary tract infections), and parasitic infections can all trigger a fever.  The body's immune response to these pathogens rele